## Dataset Information

Develop a Deep learning program to identify when an article might be fake news.


### Attributes
- id: unique id for a news article
- title: the title of a news article
- author: author of the news article
- text: the text of the article; could be incomplete
- label: a label that marks the article as potentially unreliable
    - 1: unreliable
    - 0: reliable

We are going to iterate through the text to decide if the article is reliable or not, because it not relavant to judge an article based on the title

## Import Modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import re
import nltk
import warnings
%matplotlib inline

warnings.filterwarnings('ignore')

## Loading the Dataset

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/DWDM_PROJECT/train.csv')
df.head() #Displays first five entries of the dataset

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df['title'][0] #Viewing the title of our first tuple

'House Dem Aide: We Didn’t Even See Comey’s Letter Until Jason Chaffetz Tweeted It'

In [ ]:
df['text'][0] #Viewing the news content (text) of our first tuple
'''
As we can see below, we have unnessesary things like 
\n,punctuations etc which must be removed in data cleaning
'''

'\nAs we can see below, we have unnessesary things like \n\n,punctuations etc which must be removed in data cleaning\n'

In [ ]:
df.info() 
'''summary includes list of all columns with their data types and the number of non-null values in each column'''

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


'summary includes list of all columns with their data types and the number of non-null values in each column'

## Data Proprocessing

In [ ]:
# drop unnecessary columns
'''
We are dropping these colums because we analyse based on the
text and label(reliable/not)
'''
df = df.drop(columns=['id', 'title', 'author'], axis=1)
# axis=1 means we are dropping the row

In [ ]:
# drop null values
df = df.dropna(axis=0)
# axis=0 means we are dropping the column

In [ ]:
len(df)

20761

remove special characters and punctuations

In [ ]:
df['clean_news'] = df['text'].str.lower() #converting everything to lowercase
df['clean_news']

0        house dem aide: we didn’t even see comey’s let...
1        ever get the feeling your life circles the rou...
2        why the truth might get you fired october 29, ...
3        videos 15 civilians killed in single us airstr...
4        print \nan iranian woman has been sentenced to...
                               ...                        
20795    rapper t. i. unloaded on black celebrities who...
20796    when the green bay packers lost to the washing...
20797    the macy’s of today grew from the union of sev...
20798    nato, russia to hold parallel exercises in bal...
20799      david swanson is an author, activist, journa...
Name: clean_news, Length: 20761, dtype: object

In [ ]:
df['clean_news'] = df['clean_news'].str.replace('[^A-Za-z0-9\s]', '') 

'''
Replacing everything apart from alphanumeric with a space 
'''

df['clean_news'] = df['clean_news'].str.replace('\n', '') #Replacing \n with a space
df['clean_news'] = df['clean_news'].str.replace('\s+', ' ')#Replacing gaps with a space
df['clean_news']

0        house dem aide we didnt even see comeys letter...
1        ever get the feeling your life circles the rou...
2        why the truth might get you fired october 29 2...
3        videos 15 civilians killed in single us airstr...
4        print an iranian woman has been sentenced to s...
                               ...                        
20795    rapper t i unloaded on black celebrities who m...
20796    when the green bay packers lost to the washing...
20797    the macys of today grew from the union of seve...
20798    nato russia to hold parallel exercises in balk...
20799     david swanson is an author activist journalis...
Name: clean_news, Length: 20761, dtype: object

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
'''
Stopwords are commonly used words (such as “the”, “a”, “an”, “in”)
which are basically are of no use for our analysis so we remove them.
'''
# remove stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')

'''We split the whole article into single word, and we are setting the condition,
if word not in stopwords then it'll form a list of such words and concatnate these into a sentance, 
using the lambda function'''

df['clean_news'] = df['clean_news'].apply(lambda x: " ".join([word for word in x.split() if word not in stop]))
df.head()

,text,label,clean_news
0,House Dem Aide: We Didn’t Even See Comey’s Let...,1,house dem aide didnt even see comeys letter ja...
1,Ever get the feeling your life circles the rou...,0,ever get feeling life circles roundabout rathe...
2,"Why the Truth Might Get You Fired October 29, ...",1,truth might get fired october 29 2016 tension ...
3,Videos 15 Civilians Killed In Single US Airstr...,1,videos 15 civilians killed single us airstrike...
4,Print \nAn Iranian woman has been sentenced to...,1,print iranian woman sentenced six years prison...


Our data is now clean so now we can proceed to perform analysis on our data

## Exploratory Data Analysis

In [ ]:
# visualize the frequent words
all_words = " ".join([sentence for sentence in df['clean_news']])#concatnating everything into a string
#genarating the wordcloud
wordcloud = WordCloud(width=800, height=500, random_state=42, max_font_size=100).generate(all_words)

# plot the graph
plt.figure(figsize=(15, 9)) #dimensions
plt.imshow(wordcloud, interpolation='bilinear') #to display data as image
plt.axis('off')
plt.show()

In [ ]:
# visualize the frequent words for genuine news
# Since we set the label to zero we get to see wordcloud for the genuine news
all_words = " ".join([sentence for sentence in df['clean_news'][df['label']==0]])

wordcloud = WordCloud(width=800, height=500, random_state=42, max_font_size=100).generate(all_words)

# plot the graph
plt.figure(figsize=(15, 9))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
# visualize the frequent words for fake news
# Since we set the label to one we get to see wordcloud for the fake news
all_words = " ".join([sentence for sentence in df['clean_news'][df['label']==1]])

wordcloud = WordCloud(width=800, height=500, random_state=42, max_font_size=100).generate(all_words)

# plot the graph
plt.figure(figsize=(15, 9))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

## Create Word Embeddings

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# tokenize text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['clean_news'])#This method creates the vocabulary index based on word frequency
word_index = tokenizer.word_index #gives us all unique words in data corpus
vocab_size = len(word_index)
vocab_size

In [ ]:
# padding data
'''
padding is important because each article has a different length, to equalize lengths it adds padding wherever needed
'''
'''
takes each word in the 'clean_news' and replaces it with its corresponding integer value from the 
word_index dictionary
'''
sequences = tokenizer.texts_to_sequences(df['clean_news'])
#Ensure that all sequences in a list have the same length, padding to be done after the article till it reaches 500 char length
padded_seq = pad_sequences(sequences, maxlen=500, padding='post', truncating='post')

In [ ]:
# create embedding index
#Glove contains vectors of words in hundred dimensions
embedding_index = {}
with open('/content/drive/MyDrive/DWDM_PROJECT/glove.6B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        #the zeroth value is the word itself
        word = values[0]
        #get all the values from 1st index
        coefs = np.asarray(values[1:], dtype='float32')
        embedding_index[word] = coefs

In [ ]:
# create embedding matrix only for the words from our dataset
embedding_matrix = np.zeros((vocab_size+1, 100))
'''
If the particular word from the dataset is'nt present in the embedding index
then, it'll return none, we need not add such cases into embedding matrix

If its not null and result is 100 we add it to the embedding matrix
'''
for word, i in word_index.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

A look at our embedding matrix,

In [ ]:
embedding_matrix[1]

## Input Split

In [ ]:
padded_seq[1]
'''
Each index maps to a word, this is 1st article, also it has the padded data
'''

In [ ]:
#Splitting data into 80-20 (train-test)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(padded_seq, df['label'], test_size=0.20, random_state=42, stratify=df['label'])

## Model Training

In [3]:
from keras.layers import LSTM, GRU,Dropout, Dense, Embedding
from keras import Sequential

model = Sequential([
    #Layer 1
    Embedding(vocab_size+1, 100, weights=[embedding_matrix], trainable=False),
    #Layer 2
    Dropout(0.2),
    #Layer 3
    LSTM(128),
    #Layer 4
    Dropout(0.2),
    #Layer 5
    Dense(256),
    #Layer 6
    Dense(1, activation='sigmoid')
])

model1 = Sequential([
    #Layer 1
    Embedding(vocab_size+1, 100, weights=[embedding_matrix], trainable=False),
    #Layer 2
    Dropout(0.2),
    #Layer 3
    GRU(128),
    #Layer 4
    Dropout(0.2),
    #Layer 5
    Dense(256),
    #Layer 6
    Dense(1, activation='sigmoid')
])

NameError: ignored

In [ ]:
#loss='binary_crossentropy' because we are classifying binary (real/fake)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')
model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
import pydot
import graphviz
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
#loss='binary_crossentropy' because we are classifying binary (real/fake)
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')
model1.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
import pydot
import graphviz
plot_model(model1, to_file='model1_plot.png', show_shapes=True, show_layer_names=True)

In [ ]:
# train the model
history = model.fit(x_train, y_train, epochs=51, batch_size=256, validation_data=(x_test, y_test))

In [ ]:
# train the model 1
history1 = model1.fit(x_train, y_train, epochs=51, batch_size=256, validation_data=(x_test, y_test))

In [ ]:
# visualize the results
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(['Train', 'Test'])
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['Train', 'Test'])
plt.show()

In [ ]:
# visualize the results
plt.plot(history1.history['accuracy'])
plt.plot(history1.history['val_accuracy'])
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend(['Train', 'Test'])
plt.show()

plt.plot(history1.history['loss'])
plt.plot(history1.history['val_loss'])
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(['Train', 'Test'])
plt.show()